In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from data_processing_utils import *

from deepexplain.tensorflow import DeepExplain
import seaborn as sb

import shap
import ipywidgets as widgets
shap.initjs()

import lime.lime_tabular
from sklearn.linear_model import HuberRegressor

# Data

In [ ]:
df=pd.read_csv('data.csv',sep=';', decimal=",", header=[0,1], index_col=0)
train_X, test_X, train_Y, test_Y = train_test_split(df.iloc[:,:-5], df.iloc[:,-5:], test_size=0.25, random_state=42)

# Model

## Init

In [ ]:
#params
batch_size = 256
STEPS_PER_EPOCH = int(train_X.shape[0]/batch_size)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
      0.001, decay_steps=STEPS_PER_EPOCH*1000, decay_rate=1, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr_schedule)
loss = tf.keras.losses.mean_squared_error

def init_fcnn():
    x1 = tf.keras.layers.Input(shape=(88,))
    x2 = tf.keras.layers.Dense(67, activation='elu',
                      kernel_initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in'),
                      bias_initializer=tf.keras.initializers.zeros(),
                      kernel_regularizer=tf.keras.regularizers.l2(0.000001))(x1)
    x3 = tf.keras.layers.Dense(46, activation='elu',
                      kernel_initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in'),
                      bias_initializer=tf.keras.initializers.zeros(),
                      kernel_regularizer=tf.keras.regularizers.l2(0.000001))(x2)
    x4 = tf.keras.layers.Dense(25, activation='elu',
                      kernel_initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in'),
                      bias_initializer=tf.keras.initializers.zeros(),
                      kernel_regularizer=tf.keras.regularizers.l2(0.000001))(x3)
    y = tf.keras.layers.Dense(5, activation='hard_sigmoid',
                      kernel_initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode='fan_in'),
                      bias_initializer=tf.keras.initializers.zeros(),
                      kernel_regularizer=tf.keras.regularizers.l2(0.000001))(x4)
    
    model = tf.keras.models.Model(inputs=x1, outputs=y, name='FCNN')
    model.compile(optimizer=optimizer, loss=loss)
    
    return model

model = init_fcnn()

## Train

In [ ]:
history = model.fit(
    zca_process(train_X).values, scale_to_0_1(train_Y).values,
    validation_data=(zca_process(test_X).values, scale_to_0_1(test_Y).values),
    batch_size=batch_size,
    epochs=2,
    #callbacks =[early_stopping],
    verbose=False,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(figsize=(12,6))

# Interprability

## LIME

In [ ]:
X = df.iloc[:,:-5]
Y = df.iloc[:, -5:]

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(zca_process(X).values, feature_names=[f'T={s[1]} K={s[0]}' for s in X.columns.values], class_names=['beta'], mode='regression', verbose=False)
exp = explainer.explain_instance(zca_process(X).iloc[0].values, model.predict, num_features=88, num_samples=1500, model_regressor=HuberRegressor(), top_labels=1)

In [ ]:
v0_lime= pd.DataFrame(exp.as_map()[0]).sort_values(0)
temp = pd.DataFrame(v0_lime[1].values.reshape(11,8))
cols = X.columns.levels[0][:-5].values
d = {}
for i in range(11):
    d[i] = f'K={cols[i]}'

temp = temp.rename(d)
    
cols = X.columns.levels[1][:-5].values
d = {}
for i in range(8):
    d[i] = f'T={cols[i]}'

temp = temp.rename(d,axis=1)

In [ ]:
plt.title(f'LIME Attributions Heat Map')
sb.heatmap(temp.abs().T)
#plt.savefig(f'lime_fcnn_heatmap.png', bbox_inches = 'tight')

## SHAP

In [ ]:
explainer = shap.KernelExplainer(model = model.predict, data =shap.kmeans(zca_process(train_X),15), link = "identity")
shap_value = explainer.shap_values(X = zca_process(test_X).head(1500), nsamples = 1500)

In [ ]:
list_of_labels = train_Y.columns.get_level_values(0).values

# Create a list of tuples so that the index of the label is what is returned
tuple_of_labels = list(zip(list_of_labels, range(len(list_of_labels))))

# Create a widget for the labels and then display the widget
current_label = widgets.Dropdown(options=tuple_of_labels,
                              value=0,
                              description='Select Label:'
                              )

# Display the dropdown list (Note: access index value with 'current_label.value')
current_label

In [ ]:
temp= zca_process(train_X).copy()
temp.columns = [f'T={i[1]} K={i[0]}' for i in temp_data.columns]

In [ ]:
print(f'Current Label Shown: {list_of_labels[current_label.value]}\n')

plt.title(list_of_labels[current_label.value])
plt.plot()
shap.summary_plot(shap_values = shap_value[current_label.value],
                  features = temp_data.head(1500),
                  )
#plt.savefig(f'{list_of_labels[current_label.value]}_fcnn_shapley.png', bbox_inches = 'tight')

### Bar plot

In [ ]:
temp = np.abs(shap_value[current_label.value]).mean(axis=0)

In [ ]:
pd.DataFrame([temp], columns =[i for i in temp.columns]).sort_values(0,axis=1).iloc[0,-20:].plot.barh(figsize=(12,8))
#plt.savefig(f'{list_of_labels[current_label.value]}_fcnn_shapley_bar.png',  bbox_inches = 'tight')

### Aggregated bar plot

In [ ]:
agg_shap = pd.DataFrame(np.abs(shap_value).mean(axis=1),columns =[i for i in temp.columns]).T

inds = agg_shap.sum(axis=1).sort_values(ascending=True)[-20:].index

agg_shap.loc[inds].plot.barh(width=0.6,stacked=True, figsize=(8,8))
plt.legend([r'$\upsilon_0$', r'$\rho$', r'$\sigma$', r'$\theta$', r'$\kappa$'], frameon=False)
plt.savefig('shap_fcnn_barh_1.png', bbox_inches='tight')

### Heat map

In [ ]:
temp = pd.DataFrame(agg_shap.sum(axis=1).values.reshape(11,8))

In [ ]:
cols = X.columns.levels[0][:-5].values
d = {}
for i in range(11):
    d[i] = f'K={cols[i]}'

temp = temp.rename(d)
    
cols = X.columns.levels[1][:-5].values
d = {}
for i in range(8):
    d[i] = f'T={cols[i]}'

temp = temp.rename(d,axis=1)

In [ ]:
plt.title(f'SHAP Attributions Heat Map')
sb.heatmap(test.abs().T)
#plt.savefig(f'shapley_fcnn_heatmap.png', bbox_inches = 'tight')

## E-LRP

In [ ]:
df=pd.read_csv('data.csv',sep=';', decimal=",", header=[0,1], index_col=0)
X = df.iloc[:,:-5]
Y = df.iloc[:, -5:]

In [ ]:
title_map = {
    'elrp': 'Epsilon-LRP'
}

xs =zca_process(X).iloc[0:1500].values
method_name = "elrp"

In [ ]:
session = tf.keras.backend.get_session()
with DeepExplain(session=session) as de:
    input_tensors = model.inputs
    fModel = tf.keras.models.Model(inputs = input_tensors, outputs = model.outputs)
    target_tensor = fModel(input_tensors)

    attributions = de.explain(method_name, target_tensor, input_tensors, [xs])
    #print ("Attributions:\n", attributions)

### Bar plot

In [ ]:
cols = [f'T={i[1]} K={i[0]}' for i in X.columns.values]
mean_attr = np.mean(attributions[0],axis=0)
attr = pd.DataFrame([mean_attr], columns=cols).abs().sort_values(0, axis=1)
attr.T.iloc[-20:].plot.barh(legend=False, figsize=(8,8))
#plt.savefig(f'{method_name}_fcnn_barh.png', bbox_inches='tight')

### Heat map

In [ ]:
heat_attr = pd.DataFrame(mean_attr.reshape(11,8))

In [ ]:
cols = X.columns.levels[0][:-5].values
d = {}
for i in range(11):
    d[i] = f'K={cols[i]}'

heat_attr = heat_attr.rename(d)
    
cols = X.columns.levels[1][:-5].values
d = {}
for i in range(8):
    d[i] = f'T={cols[i]}'
    
heat_attr = heat_attr.rename(d,axis=1)

In [ ]:
plt.title(f'{title_map[method_name]} Attributions Heat Map')
sb.heatmap(heat_attr.abs().T)
#plt.savefig(f'{method_name}_fcnn_heatmap.png', bbox_inches = 'tight')

## Integrated Gradient

In [ ]:
df=pd.read_csv('data.csv',sep=';', decimal=",", header=[0,1], index_col=0)
X = df.iloc[:,:-5]
Y = df.iloc[:, -5:]

In [ ]:
title_map = {
    'intgrad': 'Integrated Gradient'
}

xs =zca_process(X).iloc[0:1500].values
method_name = "intgrad"

In [ ]:
session = tf.keras.backend.get_session()
with DeepExplain(session=session) as de:
    input_tensors = model.inputs
    fModel = tf.keras.models.Model(inputs = input_tensors, outputs = model.outputs)
    target_tensor = fModel(input_tensors)

    attributions = de.explain(method_name, target_tensor, input_tensors, [xs])
    #print ("Attributions:\n", attributions)

In [28]:
### Bar plot

In [ ]:
cols = [f'T={i[1]} K={i[0]}' for i in X.columns.values]
mean_attr = np.mean(attributions[0],axis=0)
attr = pd.DataFrame([mean_attr], columns=cols).abs().sort_values(0, axis=1)
attr.T.iloc[-20:].plot.barh(legend=False, figsize=(8,8))
#plt.savefig(f'{method_name}_fcnn_barh.png', bbox_inches='tight')

In [29]:
### Heat map

In [ ]:
heat_attr = pd.DataFrame(mean_attr.reshape(11,8))

In [ ]:
cols = X.columns.levels[0][:-5].values
d = {}
for i in range(11):
    d[i] = f'K={cols[i]}'

heat_attr = heat_attr.rename(d)
    
cols = X.columns.levels[1][:-5].values
d = {}
for i in range(8):
    d[i] = f'T={cols[i]}'
    
heat_attr = heat_attr.rename(d,axis=1)

In [ ]:
plt.title(f'{title_map[method_name]} Attributions Heat Map')
sb.heatmap(heat_attr.abs().T)
#plt.savefig(f'{method_name}_fcnn_heatmap.png', bbox_inches = 'tight')